# Ejemplos de uso de Numba

Basándome en: https://numba.pydata.org/numba-doc/0.15.1/quickstart.html

In [1]:
import numpy as np
from numba import jit
import cProfile

@jit
def sum(x, y):
    return x + y

In [2]:
@jit('f8(f8,f8)')
def sum_float(x, y):
    return x + y

In [3]:
sum_float(2,0)

2.0

In [4]:
def sum1d_iterative(array):
    sum = 0.0
    for i in range(array.shape[0]):
        sum += array[i]
    return sum

In [5]:
@jit('f8(f8[:])')
def sum1d(array):
    sum = 0.0
    for i in range(array.shape[0]):
        sum += array[i]
    return sum

In [6]:
test_array = np.ones(1000000)

In [7]:
cProfile.run('sum1d_iterative(test_array)')

         4 function calls in 0.470 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.470    0.470    0.470    0.470 <ipython-input-4-3415b2b468cf>:1(sum1d_iterative)
        1    0.000    0.000    0.470    0.470 <string>:1(<module>)
        1    0.000    0.000    0.470    0.470 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [8]:
cProfile.run('sum1d(test_array)')

         139 function calls (127 primitive calls) in 0.002 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.001    0.001    0.001    0.001 <ipython-input-5-8c091ed70b8c>:1(sum1d)
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
        6    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        2    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     12/6    0.000    0.000    0.000    0.000 abstract.py:118(__hash__)
        3    0.000    0.000    0.000    0.000 abstract.py:121(__eq__)
        1    0.000    0.000    0.000    0.000 abstract.py:148(can_convert_from)
        1    0.000    0.000    0.000    0.000 abstract.py:19(_autoincr)
        2    0.000    0.000    0.000    0.000 abstract.py:48(_intern)
        2   

In [9]:
cProfile.run('np.sum(test_array)')

         12 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 <__array_function__ internals>:2(sum)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:2100(_sum_dispatcher)
        1    0.000    0.000    0.001    0.001 fromnumeric.py:2105(sum)
        1    0.000    0.000    0.001    0.001 fromnumeric.py:70(_wrapreduction)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:71(<dictcomp>)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.001    0.001 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.00

# Diferentes tipos de compilación

In [10]:
def min2x2(matrix):
    shape = matrix.shape
    output_matrix = np.zeros((shape[0]//2,shape[1]//2))
    for x in range(0,shape[1],2):
        for y in range(0,shape[0],2):
            output_matrix[y//2][x//2] = np.amin(matrix[y:y+2,x:x+2])
    return output_matrix

In [11]:
random_matrix = np.random.rand(5000,5000)

In [12]:
random_matrix[0:2,0:2]

array([[0.53110803, 0.35774309],
       [0.41192535, 0.28125347]])

In [13]:
cProfile.run('minimized = min2x2(random_matrix)')

         50000005 function calls in 111.539 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  6250000    6.082    0.000   88.831    0.000 <__array_function__ internals>:2(amin)
        1   22.708   22.708  111.539  111.539 <ipython-input-10-4eea0400048b>:1(min2x2)
        1    0.000    0.000  111.539  111.539 <string>:1(<module>)
  6250000    1.246    0.000    1.246    0.000 fromnumeric.py:2709(_amin_dispatcher)
  6250000   10.736    0.000   75.708    0.000 fromnumeric.py:2714(amin)
  6250000   12.986    0.000   64.972    0.000 fromnumeric.py:70(_wrapreduction)
  6250000    4.139    0.000    4.139    0.000 fromnumeric.py:71(<dictcomp>)
        1    0.000    0.000  111.539  111.539 {built-in method builtins.exec}
  6250000    5.795    0.000   81.502    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        1    0.000   

In [14]:
minimized[0,0]

0.2812534673712238

In [15]:
from numba import f8, jit

jitmin2x2 = jit(f8[:,:](f8[:,:]))(min2x2)

In [16]:
cProfile.run('minimized_jit = jitmin2x2(random_matrix)')

         139 function calls (127 primitive calls) in 1.012 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    1.011    1.011    1.011    1.011 <ipython-input-10-4eea0400048b>:1(min2x2)
        1    0.000    0.000    1.012    1.012 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
        6    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        2    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     12/6    0.000    0.000    0.000    0.000 abstract.py:118(__hash__)
        4    0.000    0.000    0.000    0.000 abstract.py:121(__eq__)
        1    0.000    0.000    0.000    0.000 abstract.py:148(can_convert_from)
        2    0.000    0.000    0.000    0.000 abstract.py:48(_intern)
        2    0.000    0.000    0.000    0.000 abstract.py:60(__call__)
        2  

In [17]:
minimized_jit[0,0]

0.2812534673712238

Claramente Numba por sí misma nos provee de las capacidades para generar código rápido facilmente. Por otro lado, habrá que compararlo con su implementación con GPUs para ver que nos ofrece.

# CUDA en Numba

Ver el ejemplo en la documentación:
https://numba.pydata.org/numba-doc/dev/cuda/examples.html

## Multiplicación de matrices simple

Utilizaremos el algoritmo ingenuo de multiplicación de matrices en con jit y con CUDA para comparar su desempeño.

In [2]:
from numba import cuda, float32
import numpy as np
from numba import jit
import cProfile

@cuda.jit
def matmul_cuda(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    #El metodo cuda.grid nos da la posición absoluta de nuestro hilo de ejecución en todo el bloque de bloques.
    i, j = cuda.grid(2)#Esto nos permite asignar un índice único por hilo en vez de utilizar fórmulas.
    if i < C.shape[0] and j < C.shape[1]:
        tmp = 0.
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]
        C[i, j] = tmp

A diferencia de las funciones optimizadas por Numba, CUDA no devuelve valores sino que los cambia.

In [19]:
A = np.ones((300,300))*2
B = np.ones((300,300))*3

C = np.zeros((300,300))

cProfile.run('matmul_cuda[(30,30),(10,10)](A,B,C)')

         227802 function calls (214860 primitive calls) in 0.389 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  524/514    0.001    0.000    0.013    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        5    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:103(release)
        5    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:143(__init__)
        5    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:147(__enter__)
        5    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:151(__exit__)
        5    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:157(_get_module_lock)
        5    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:176(cb)
     10/5    0.000    0.000    0.012    0.002 <frozen importlib._bootstrap>:211(_call_with_frames_removed)
       35    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:

        1    0.000    0.000    0.000    0.000 builder.py:526(not_)
        1    0.000    0.000    0.000    0.000 builder.py:537(neg)
       18    0.000    0.000    0.000    0.000 builder.py:548(_icmp)
       17    0.000    0.000    0.000    0.000 builder.py:559(icmp_signed)
        1    0.000    0.000    0.000    0.000 builder.py:568(icmp_unsigned)
        8    0.000    0.000    0.000    0.000 builder.py:609(select)
       65    0.000    0.000    0.001    0.000 builder.py:717(alloca)
      206    0.001    0.000    0.004    0.000 builder.py:735(load)
      212    0.001    0.000    0.004    0.000 builder.py:748(store)
        7    0.000    0.000    0.000    0.000 builder.py:805(branch)
        6    0.000    0.000    0.000    0.000 builder.py:813(cbranch)
        1    0.000    0.000    0.000    0.000 builder.py:830(ret_void)
        2    0.000    0.000    0.000    0.000 builder.py:837(ret)
        7    0.000    0.000    0.000    0.000 builder.py:854(call)
       59    0.000    0.000    0.

        1    0.000    0.000    0.154    0.154 compiler_machinery.py:318(run)
        2    0.000    0.000    0.000    0.000 compiler_machinery.py:34(__init__)
        4    0.000    0.000    0.001    0.000 compiler_machinery.py:343(dependency_analysis)
       42    0.000    0.000    0.000    0.000 compiler_machinery.py:358(resolve_requires)
        2    0.000    0.000    0.000    0.000 compiler_machinery.py:359(walk)
       66    0.000    0.000    0.000    0.000 compiler_machinery.py:39(name)
       88    0.000    0.000    0.000    0.000 compiler_machinery.py:402(is_registered)
       66    0.000    0.000    0.000    0.000 compiler_machinery.py:405(get)
       22    0.000    0.000    0.000    0.000 compiler_machinery.py:67(analysis)
       22    0.000    0.000    0.000    0.000 compiler_machinery.py:74(run_initialization)
       22    0.000    0.000    0.000    0.000 compiler_machinery.py:89(run_finalizer)
       40    0.000    0.000    0.000    0.000 compiler_machinery.py:96(get_analysi

        1    0.000    0.000    0.000    0.000 cudadecl.py:92(Cuda_syncthreads_count)
        1    0.000    0.000    0.000    0.000 cudadecl.py:98(Cuda_syncthreads_and)
        1    0.000    0.000    0.001    0.001 cudaimpl.py:1(<module>)
        4    0.000    0.000    0.000    0.000 cudaimpl.py:477(gen_deg_rad)
        5    0.000    0.000    0.000    0.000 cudaimpl.py:507(_atomic_dispatcher)
        1    0.000    0.000    0.001    0.001 cudaimpl.py:75(cuda_grid)
        1    0.000    0.000    0.003    0.003 cudamath.py:1(<module>)
        1    0.000    0.000    0.000    0.000 cudamath.py:10(Math_unary)
        1    0.000    0.000    0.000    0.000 cudamath.py:47(Math_atan2)
        1    0.000    0.000    0.000    0.000 cudamath.py:58(Math_hypot)
        1    0.000    0.000    0.000    0.000 cudamath.py:69(Math_binary)
        1    0.000    0.000    0.000    0.000 cudamath.py:78(Math_pow)
        1    0.000    0.000    0.000    0.000 cudamath.py:88(Math_isnan)
        1    0.000    0.00

        2    0.000    0.000    0.000    0.000 inline_closurecall.py:281(__init__)
        2    0.000    0.000    0.000    0.000 inline_closurecall.py:299(check)
        2    0.000    0.000    0.000    0.000 inline_closurecall.py:308(<listcomp>)
        2    0.000    0.000    0.000    0.000 inline_closurecall.py:330(<listcomp>)
        1    0.000    0.000    0.000    0.000 inline_closurecall.py:62(__init__)
        1    0.000    0.000    0.006    0.006 inline_closurecall.py:69(run)
       11    0.000    0.000    0.000    0.000 inline_closurecall.py:746(_make_debug_print)
        3    0.000    0.000    0.000    0.000 inline_closurecall.py:747(debug_print)
        1    0.000    0.000    0.000    0.000 inline_closurecall.py:753(_debug_dump)
        1    0.000    0.000    0.000    0.000 inline_closurecall.py:825(_find_arraycall)
        1    0.000    0.000    0.000    0.000 inline_closurecall.py:906(_inline_arraycall)
       43    0.000    0.000    0.000    0.000 inspect.py:158(isfunction)


        4    0.000    0.000    0.000    0.000 itanium_mangler.py:97(<genexpr>)
        1    0.000    0.000    0.000    0.000 libdevice.py:1(<module>)
        2    0.000    0.000    0.000    0.000 libdevice.py:126(modf_implement)
        4    0.000    0.000    0.000    0.000 libdevice.py:13(bool_implement)
       50    0.000    0.000    0.000    0.000 libdevice.py:27(unary_implement)
       10    0.000    0.000    0.000    0.000 libdevice.py:38(binary_implement)
        2    0.000    0.000    0.000    0.000 libdevice.py:49(powi_implement)
        2    0.000    0.000    0.000    0.000 libs.py:27(get_libdevice)
        1    0.000    0.000    0.001    0.001 libs.py:33(open_libdevice)
        2    0.000    0.000    0.007    0.004 libs.py:38(get_cudalib)
        2    0.000    0.000    0.008    0.004 libs.py:54(open_cudalib)
       10    0.000    0.000    0.000    0.000 linecache.py:37(getlines)
        3    0.000    0.000    0.001    0.000 linker.py:5(link_modules)
        2    0.000    0.00

        1    0.001    0.001    0.001    0.001 runtime.py:76(safe_cuda_api_call)
        1    0.000    0.000    0.000    0.000 runtime.py:83(_check_error)
        1    0.000    0.000    0.000    0.000 runtime.py:90(_find_api)
        9    0.000    0.000    0.000    0.000 scalars.py:32(__init__)
        9    0.000    0.000    0.000    0.000 scalars.py:78(__init__)
        2    0.000    0.000    0.000    0.000 scalars.py:89(can_convert_to)
        1    0.000    0.000    0.000    0.000 sigutils.py:17(normalize_signature)
        3    0.000    0.000    0.000    0.000 sigutils.py:38(check_type)
        6    0.000    0.000    0.000    0.000 slicing.py:15(fix_index)
        8    0.000    0.000    0.000    0.000 sre_compile.py:249(_compile_charset)
        8    0.000    0.000    0.000    0.000 sre_compile.py:276(_optimize_charset)
       10    0.000    0.000    0.000    0.000 sre_compile.py:423(_simple)
        3    0.000    0.000    0.000    0.000 sre_compile.py:432(_generate_overlap_table)
  

       18    0.000    0.000    0.000    0.000 typeof.py:37(typeof_impl)
       18    0.000    0.000    0.000    0.000 typeof.py:58(_typeof_buffer)
        4    0.000    0.000    0.000    0.000 typeof.py:82(typeof_type)
       14    0.000    0.000    0.000    0.000 types.py:105(_to_string)
      152    0.000    0.000    0.000    0.000 types.py:116(__init__)
      124    0.000    0.000    0.000    0.000 types.py:121(_to_string)
       82    0.000    0.000    0.000    0.000 types.py:127(__eq__)
       59    0.000    0.000    0.000    0.000 types.py:137(gep)
        1    0.000    0.000    0.000    0.000 types.py:155(_to_string)
        9    0.000    0.000    0.000    0.000 types.py:170(__init__)
        1    0.000    0.000    0.000    0.000 types.py:175(_to_string)
        1    0.000    0.000    0.000    0.000 types.py:177(<listcomp>)
       40    0.000    0.000    0.000    0.000 types.py:205(__new__)
      321    0.000    0.000    0.000    0.000 types.py:231(__eq__)
      184    0.000    

      956    0.000    0.000    0.000    0.000 {method 'insert' of 'list' objects}
       52    0.000    0.000    0.000    0.000 {method 'isalpha' of 'str' objects}
       31    0.000    0.000    0.000    0.000 {method 'isdigit' of 'str' objects}
       78    0.000    0.000    0.000    0.000 {method 'isidentifier' of 'str' objects}
        1    0.000    0.000    0.000    0.000 {method 'items' of 'collections.OrderedDict' objects}
      775    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
        9    0.000    0.000    0.000    0.000 {method 'items' of 'mappingproxy' objects}
     1662    0.001    0.000    0.004    0.000 {method 'join' of 'str' objects}
      174    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
       28    0.000    0.000    0.000    0.000 {method 'lower' of 'str' objects}
     1148    0.000    0.000    0.000    0.000 {method 'lstrip' of 'str' objects}
     5934    0.002    0.000    0.002    0.000 {method 'match' of 're.Pattern'

In [20]:
C

array([[1800., 1800., 1800., ..., 1800., 1800., 1800.],
       [1800., 1800., 1800., ..., 1800., 1800., 1800.],
       [1800., 1800., 1800., ..., 1800., 1800., 1800.],
       ...,
       [1800., 1800., 1800., ..., 1800., 1800., 1800.],
       [1800., 1800., 1800., ..., 1800., 1800., 1800.],
       [1800., 1800., 1800., ..., 1800., 1800., 1800.]])

In [21]:
C = np.zeros((300,300))

In [22]:
@jit
def matmul_jit(A, B, C):
    """Perform square matrix multiplication of C = A * B
    """
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            tmp = 0.
            for k in range(A.shape[1]):
                tmp += A[i, k] * B[k, j]
            C[i, j] = tmp

In [38]:
cProfile.run('matmul_jit(A,B,C)')

         1123 function calls (1102 primitive calls) in 0.016 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.016    0.016 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
       14    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(info)
       15    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        2    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       11    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        5    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     10/6    0.000    0.000    0.000    0.000 abstract.py:118(__hash_

        3    0.000    0.000    0.000    0.000 {method 'format' of 'str' objects}
       11    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
        2    0.000    0.000    0.000    0.000 {method 'indices' of 'slice' objects}
        1    0.000    0.000    0.000    0.000 {method 'items' of 'mappingproxy' objects}
        1    0.000    0.000    0.000    0.000 {method 'join' of 'str' objects}
        4    0.000    0.000    0.000    0.000 {method 'pop' of 'dict' objects}
        4    0.000    0.000    0.000    0.000 {method 'rpartition' of 'str' objects}
        2    0.000    0.000    0.000    0.000 {method 'squeeze' of 'numpy.ndarray' objects}
        1    0.000    0.000    0.000    0.000 {method 'update' of 'dict' objects}




In [24]:
C

array([[1800., 1800., 1800., ..., 1800., 1800., 1800.],
       [1800., 1800., 1800., ..., 1800., 1800., 1800.],
       [1800., 1800., 1800., ..., 1800., 1800., 1800.],
       ...,
       [1800., 1800., 1800., ..., 1800., 1800., 1800.],
       [1800., 1800., 1800., ..., 1800., 1800., 1800.],
       [1800., 1800., 1800., ..., 1800., 1800., 1800.]])

Incluso esta matriz relativamente pequeña se genera más rápido en CUDA. Es claro que para problemas más grandes, esta tendencia se mantendrá.

### Prueba simple: Método Monte Carlo

Este se encuentra en la documentación de Numba y es sumamente ilustrativo.

In [3]:
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float64


@cuda.jit
def estimar_pi(rng_states, iteraciones, salida):
    thread_id = cuda.grid(1)
    
    acumulador = 0
    for i in range(iteraciones):
        x = xoroshiro128p_uniform_float64(rng_states, thread_id)
        y = xoroshiro128p_uniform_float64(rng_states, thread_id)
        if x**2 + y**2 <= 1.0:
            acumulador += 1

    salida[thread_id] = 4.0 * acumulador / iteraciones

In [4]:
threads_per_block = 64
blocks = 24
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=42)
out = np.zeros(threads_per_block * blocks, dtype=np.float64)

In [6]:
cProfile.run('estimar_pi[blocks, threads_per_block](rng_states, 10000, out)')

         1123 function calls (1102 primitive calls) in 0.005 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.005    0.005 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
       14    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(info)
       15    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        2    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       11    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        5    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     10/6    0.000    0.000    0.000    0.000 abstract.py:118(__hash_

In [7]:
print('pi:', out.mean())

pi: 3.141770052083333


In [8]:
threads_per_block = 256
blocks = 100
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=42)
out = np.zeros(threads_per_block * blocks, dtype=np.float64)

In [9]:
cProfile.run('estimar_pi[blocks, threads_per_block](rng_states, 10000, out)')

         1123 function calls (1102 primitive calls) in 0.014 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.014    0.014 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
       14    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(info)
       15    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        2    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       11    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        5    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     10/6    0.000    0.000    0.000    0.000 abstract.py:118(__hash_

In [10]:
print('pi:', out.mean())

pi: 3.141607890625


In [11]:
threads_per_block = 1024
blocks = 1024
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=42)
out = np.zeros(threads_per_block * blocks, dtype=np.float64)

In [12]:
cProfile.run('estimar_pi[blocks, threads_per_block](rng_states, 100000, out)')

         1123 function calls (1102 primitive calls) in 3.991 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    3.991    3.991 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
       14    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(info)
       15    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        2    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       11    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        5    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     10/6    0.000    0.000    0.000    0.000 abstract.py:118(__hash_

In [13]:
print('pi:', out.mean())

pi: 3.141599155731203


### Reducción

In [14]:
@cuda.reduce
def sum_reduce(a, b):#float32
    return a + b

#sum_reduce(arreglo):
    #recursivamente aplica sum_reduce(a,b) a los elementos hasta obtener la reduccion

# [1,2,3,4,5..] -> Respuesta (1)

#[1,2,3,4,5,6]
#[1,2]-> 3
#[3,4]-> 7
#[5,6]-> 11

#Segunda pasada

#[3,7,11]
#[3,7] -> 10
#[11] -> 11

# Tercera pasada

#[10,11] -> 21 que es la reducción final

In [15]:
threads_per_block = 1024
blocks = 1024
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=42)
out = cuda.device_array(threads_per_block * blocks, dtype=np.float64)

Notar que en este caso estamos creando el arreglo de salida directamente en el dispositivo, lo que nos ahorra tiempos de transmisión considerables.

In [16]:
retorno = cuda.device_array(1, dtype=np.float64)

cProfile.run('estimar_pi[blocks, threads_per_block](rng_states, 100000, out)')

         443 function calls (427 primitive calls) in 0.002 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
        5    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        5    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        2    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       11    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        5    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     10/6    0.000    0.000    0.000    0.000 abstract.py:118(__hash__)
        3    0.000    0.000    0.000    0.000 abstract.py:121(__eq__

In [18]:
cProfile.run('sum_reduce(out, res=retorno)')

         2342 function calls (2303 primitive calls) in 0.007 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.007    0.007 <string>:1(<module>)
        8    0.000    0.000    0.000    0.000 <string>:1(__new__)
       24    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(info)
       25    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        4    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       32    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        4    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
    20/12    0.000    0.000    0.000    0.000 abstract.py:118(__hash_

In [19]:
retorno[0]/(threads_per_block * blocks)

3.1415991557312015

En los ejemplos previos se realizaba la reducción en el anfitrión una vez se regresaba el arreglo. Esto implicaba la transmisión de todo el arreglo al anfitrión (y esto puede tomar mucho tiempo) por lo que también tenemos la facilidad para generar una reducción en la GPU y simplemente obtener los resultados.

### Operaciones atómicas

In [20]:
from numba.cuda.random import xoroshiro128p_normal_float64

#[1]

@cuda.jit
def conteo_aleatorios(rng_states, decision, iters, salida):
    thread_id = cuda.grid(1)

    for i in range(iters):
        x = xoroshiro128p_normal_float64(rng_states, thread_id)
        if(x > decision or -x > decision):
            cuda.atomic.add(salida, 0, 1)
            
#0.5
#0.1, 0.4, 1.2, -4.0, 0.3

In [28]:
threads_per_block = 1024
blocks = 1024
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=100)
out = np.zeros(1, dtype=np.int32)
barrera_decision = 0.1
iters = 500

In [29]:
cProfile.run('conteo_aleatorios[blocks, threads_per_block](rng_states, barrera_decision, iters, out)')

         1089 function calls (1068 primitive calls) in 0.105 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.105    0.105 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
       14    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(info)
       15    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        2    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       14    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        5    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     11/7    0.000    0.000    0.000    0.000 abstract.py:118(__hash_

In [30]:
print("De los {:,} ensayos, {:,} muestras aleatorias ({}%) de una normal estandar tienen valor absoluto mayor a {}".
     format(threads_per_block * blocks * iters, out[0], out[0]*100/(threads_per_block * blocks * iters), barrera_decision))

De los 524,288,000 ensayos, 482,519,266 muestras aleatorias (92.0332462310791%) de una normal estandar tienen valor absoluto mayor a 0.1


In [31]:
out = np.zeros(1, dtype=np.int32)
barrera_decision = 5.0

In [32]:
cProfile.run('conteo_aleatorios[blocks, threads_per_block](rng_states, barrera_decision, iters, out)')

         1089 function calls (1068 primitive calls) in 0.100 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.100    0.100 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
       14    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(info)
       15    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        2    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       14    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        5    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     11/7    0.000    0.000    0.000    0.000 abstract.py:118(__hash_

In [33]:
print("De los {:,} ensayos, {:,} muestras aleatorias ({}%) de una normal estandar tienen valor absoluto mayor a {}".
     format(threads_per_block * blocks * iters, out[0], out[0]*100/(threads_per_block * blocks * iters), barrera_decision))

De los 524,288,000 ensayos, 310 muestras aleatorias (5.91278076171875e-05%) de una normal estandar tienen valor absoluto mayor a 5.0


Aunque las variables atómicas permiten simplificar el código, hay que tomar en cuenta que su uso implica un tiempo de espera para verificar el acceso atómico al espacio de memoria.

Debido a lo anterior, es conveniente utilizarlas solo si se sabe a priori que van a ser accesadas poco. De lo contrario, tal vez sea más eficiente buscar otra metodología (como utilizar varios kernels y guardar resultados intermedios).

#### Perspectiva: Guardando resultados en memoria

In [34]:
@cuda.jit
def conteo_aleatorios_memoria(rng_states, decision, iters, salida):
    thread_id = cuda.grid(1)
    skip = cuda.gridsize(1)
    for i in range(iters):
        x = xoroshiro128p_normal_float64(rng_states, thread_id)
        if(x > decision or -x > decision):
            salida[thread_id+(i*skip)] = 1
        else:
            salida[thread_id+(i*skip)] = 0

In [35]:
threads_per_block = 1024
blocks = 1024
rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=100)
iters = 500 #Nota que con 10,000 se le acaba la memoria al equipo que tenemos disponible.
out = cuda.device_array(threads_per_block * blocks * iters, dtype=np.float64)
barrera_decision = 0.1

In [37]:
cProfile.run('conteo_aleatorios_memoria[blocks, threads_per_block](rng_states, barrera_decision, iters, out)')

         473 function calls (457 primitive calls) in 0.002 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
        5    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        5    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        2    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       14    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        5    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     11/7    0.000    0.000    0.000    0.000 abstract.py:118(__hash__)
        4    0.000    0.000    0.000    0.000 abstract.py:121(__eq__

In [38]:
retorno = cuda.device_array(1, dtype=np.float64)
cProfile.run('sum_reduce(out, res=retorno)')

         2342 function calls (2303 primitive calls) in 0.005 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.005    0.005 <string>:1(<module>)
        8    0.000    0.000    0.000    0.000 <string>:1(__new__)
       24    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(info)
       25    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        4    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       32    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        4    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
    20/12    0.000    0.000    0.000    0.000 abstract.py:118(__hash_

In [39]:
r1 = retorno[0]

In [40]:
del(out)

In [41]:
out = cuda.device_array(threads_per_block * blocks * iters, dtype=np.float64)
barrera_decision = 5.0
cProfile.run('conteo_aleatorios_memoria[blocks, threads_per_block](rng_states, barrera_decision, iters, out)')

         473 function calls (457 primitive calls) in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        4    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 <string>:1(__new__)
        5    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        5    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        2    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       14    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        5    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
     11/7    0.000    0.000    0.000    0.000 abstract.py:118(__hash__)
        4    0.000    0.000    0.000    0.000 abstract.py:121(__eq__

In [42]:
retorno = cuda.device_array(1, dtype=np.float64)
cProfile.run('sum_reduce(out, res=retorno)')

         2342 function calls (2303 primitive calls) in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        9    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        8    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:389(parent)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        8    0.000    0.000    0.000    0.000 <string>:1(__new__)
       24    0.000    0.000    0.000    0.000 __init__.py:1412(debug)
        1    0.000    0.000    0.000    0.000 __init__.py:1424(info)
       25    0.000    0.000    0.000    0.000 __init__.py:1677(isEnabledFor)
        4    0.000    0.000    0.000    0.000 abc.py:100(__subclasscheck__)
       32    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        4    0.000    0.000    0.000    0.000 abstract.py:115(__repr__)
    20/12    0.000    0.000    0.000    0.000 abstract.py:118(__hash_

In [43]:
r2 = retorno[0]

In [44]:
print("De los {:,} ensayos, {:,} muestras aleatorias ({}%) de una normal estandar tienen valor absoluto mayor a {}".
     format(threads_per_block * blocks * iters, r1, r1*100/(threads_per_block * blocks * iters), 0.1))

De los 524,288,000 ensayos, 482,536,270.0 muestras aleatorias (92.03648948669434%) de una normal estandar tienen valor absoluto mayor a 0.1


In [45]:
print("De los {:,} ensayos, {:,} muestras aleatorias ({}%) de una normal estandar tienen valor absoluto mayor a {}".
     format(threads_per_block * blocks * iters, r2, r2*100/(threads_per_block * blocks * iters), 5.0))

De los 524,288,000 ensayos, 282.0 muestras aleatorias (5.37872314453125e-05%) de una normal estandar tienen valor absoluto mayor a 5.0


Como se puede notar, el uso de operaciones atómicas disminuye fuertemente el desempeño en tiempo del alogoritmo. Sin embargo, para casos donde se quiera utilizar más memoria (por ejemplo 10,000 iteraciones por kernel), el guardar los resultados intermedios como se hizo aquí puede resultar inviable.

#### Perspectiva: Tiempo de procesamiento con vectorización:

In [127]:
it_tot = 1024 * 1024 * 500

In [128]:
def conteo_aleatorios_vec(decision, iters):
    x = np.random.normal(loc = 0.0, scale = 1.0, size=iters)
    return np.sum(x > decision) + np.sum(x < -decision)

In [129]:
res = []
def wrapper_resultados(r):
    res.append(r)

In [130]:
cProfile.run('wrapper_resultados(conteo_aleatorios_vec(0.1, it_tot))')

         25 function calls in 28.120 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    1.020    0.510 <__array_function__ internals>:2(sum)
        1    0.986    0.986   28.109   28.109 <ipython-input-128-f4ea28862e49>:1(conteo_aleatorios_vec)
        1    0.000    0.000    0.000    0.000 <ipython-input-129-a51952decef1>:2(wrapper_resultados)
        1    0.010    0.010   28.120   28.120 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:2100(_sum_dispatcher)
        2    0.000    0.000    1.020    0.510 fromnumeric.py:2105(sum)
        2    0.000    0.000    1.020    0.510 fromnumeric.py:70(_wrapreduction)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:71(<dictcomp>)
        1    0.000    0.000   28.120   28.120 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        2    0.000    0.

In [131]:
cProfile.run('wrapper_resultados(conteo_aleatorios_vec(5.0, it_tot))')

         25 function calls in 27.729 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    1.019    0.509 <__array_function__ internals>:2(sum)
        1    0.988    0.988   27.719   27.719 <ipython-input-128-f4ea28862e49>:1(conteo_aleatorios_vec)
        1    0.000    0.000    0.000    0.000 <ipython-input-129-a51952decef1>:2(wrapper_resultados)
        1    0.010    0.010   27.729   27.729 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:2100(_sum_dispatcher)
        2    0.000    0.000    1.019    0.509 fromnumeric.py:2105(sum)
        2    0.000    0.000    1.019    0.509 fromnumeric.py:70(_wrapreduction)
        2    0.000    0.000    0.000    0.000 fromnumeric.py:71(<dictcomp>)
        1    0.000    0.000   27.729   27.729 {built-in method builtins.exec}
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        2    0.000    0.

In [132]:
print("De los {:,} ensayos, {:,} muestras aleatorias ({}%) de una normal estandar tienen valor absoluto mayor a {}".
     format(it_tot, res[0], res[0]*100/it_tot, 0.1))

De los 524,288,000 ensayos, 482,515,542 muestras aleatorias (92.03253593444825%) de una normal estandar tienen valor absoluto mayor a 0.1


In [133]:
print("De los {:,} ensayos, {:,} muestras aleatorias ({}%) de una normal estandar tienen valor absoluto mayor a {}".
     format(it_tot, res[1], res[1]*100/it_tot, 5.0))

De los 524,288,000 ensayos, 304 muestras aleatorias (5.79833984375e-05%) de una normal estandar tienen valor absoluto mayor a 5.0
